In [1]:
from openai import OpenAI
import json
import json
from typing import Union
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
)

In [2]:
# 定义查询鲜花库存的函数
def get_flower_inventory(city):
    """获取指定城市的鲜花库存"""
    if "北京" in city:
        return json.dumps({"city": "北京", "inventory": "玫瑰: 100, 郁金香: 150"})
    elif "上海" in city:
        return json.dumps({"city": "上海", "inventory": "百合: 80, 康乃馨: 120"})
    elif "深圳" in city:
        return json.dumps({"city": "深圳", "inventory": "向日葵: 200, 玉兰: 90"})
    else:
        return json.dumps({"city": city, "inventory": "未知"})
# 定义工具列表（函数元数据）
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flower_inventory",
            "description": "获取指定城市的鲜花库存",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称，如北京、上海或深圳"
                    }
                },
                "required": ["city"]
            }
        }
    }
]


In [3]:
# 初始化对话内容
messages = [{"role": "user", "content": "北京、上海和深圳的鲜花库存是多少？"}]
print("message:", messages)
# 第一次对话响应
first_response = client.chat.completions.create(
    model="qwen25:72b",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
# 打印响应的内容
print(first_response)
response_message = first_response.choices[0].message

message: [{'role': 'user', 'content': '北京、上海和深圳的鲜花库存是多少？'}]
ChatCompletion(id='chat-f1d204ffc9684bdd9cc19085f232ffda', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-124ea2fa27fd47d4b375bc53055062df', function=Function(arguments='{"city": "\\u5317\\u4eac"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='chatcmpl-tool-c7b7b7d8f0994252b16135a8a836b034', function=Function(arguments='{"city": "\\u4e0a\\u6d77"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='chatcmpl-tool-47d98eff886c44ba81d9162334e55eac', function=Function(arguments='{"city": "\\u6df1\\u5733"}', name='get_flower_inventory'), type='function')]), stop_reason=None)], created=1735788147, model='qwen25:72b', object='chat.completion', service_tier=None, system_fingerprint=

In [4]:
# 检查是否需要调用工具
tool_calls = response_message.tool_calls
if tool_calls:
    messages.append(response_message)
# 如果需要调用工具，调用工具并添加库存查询结果
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function_response = get_flower_inventory(
            city=function_args.get("city")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
# 打印当前消息列表
print("message:", messages)

message: [{'role': 'user', 'content': '北京、上海和深圳的鲜花库存是多少？'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-124ea2fa27fd47d4b375bc53055062df', function=Function(arguments='{"city": "\\u5317\\u4eac"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='chatcmpl-tool-c7b7b7d8f0994252b16135a8a836b034', function=Function(arguments='{"city": "\\u4e0a\\u6d77"}', name='get_flower_inventory'), type='function'), ChatCompletionMessageToolCall(id='chatcmpl-tool-47d98eff886c44ba81d9162334e55eac', function=Function(arguments='{"city": "\\u6df1\\u5733"}', name='get_flower_inventory'), type='function')]), {'tool_call_id': 'chatcmpl-tool-124ea2fa27fd47d4b375bc53055062df', 'role': 'tool', 'name': 'get_flower_inventory', 'content': '{"city": "\\u5317\\u4eac", "inventory": "\\u73ab\\u7470: 100, \\u90c1\\u91d1\\u9999: 150"}'}, {'tool_call_id': 'chatcmpl-tool-c7b7b7

In [5]:
# 第二次向大模型发送对话以获取最终响应
second_response = client.chat.completions.create(
    model="qwen25:72b",
    messages=messages
)
# 打印最终响应
final_response = second_response.choices[0].message
print(final_response)

ChatCompletionMessage(content='以下是北京、上海和深圳的鲜花库存情况：\n\n- **北京**:\n  - 玫瑰: 100\n  - 铁线莲: 150\n\n- **上海**:\n  - 百合: 80\n  - 康乃馨: 120\n\n- **深圳**:\n  - 向日葵: 200\n  - 兰花: 90\n\n这些数据可以帮助您了解当前各城市的鲜花供应情况。如果您需要更多信息或有其他需求，欢迎随时询问！', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[])
